In [1]:
from google.colab import files

# Hochladen der Datei vom lokalen Computer
uploaded = files.upload()

Saving inflation_cleaned.csv to inflation_cleaned.csv
Saving kauf_cleaned.csv to kauf_cleaned.csv
Saving wohnen_cleaned.csv to wohnen_cleaned.csv
Saving zins_cleaned.csv to zins_cleaned.csv


In [2]:
# 📥 Schritt 1: CSV-Dateien laden
# Wir importieren die bereinigten CSV-Dateien mit Zeitindex (Datumswerte als Index setzen)
# Wichtig: parse_dates stellt sicher, dass der Index als Zeitreihe erkannt wird
import pandas as pd

df_zins = pd.read_csv("zins_cleaned.csv", index_col=0, parse_dates=True)
df_inflation = pd.read_csv("inflation_cleaned.csv", index_col=0, parse_dates=True)
df_wohnen = pd.read_csv("wohnen_cleaned.csv", index_col=0, parse_dates=True)
df_kauf = pd.read_csv("kauf_cleaned.csv", index_col=0, parse_dates=True)

In [3]:
# 📊 Schritt 2: Zeitindex kontrollieren
# Bevor wir mergen, prüfen wir, ob alle Zeitreihen die gleichen Zeitbereiche abdecken.
print("🕓 Zeiträume der vier Datensätze:")
print("Zinsen:     ", df_zins.index.min(), "→", df_zins.index.max())
print("Inflation:  ", df_inflation.index.min(), "→", df_inflation.index.max())
print("Wohnen:     ", df_wohnen.index.min(), "→", df_wohnen.index.max())
print("Kaufpreis:  ", df_kauf.index.min(), "→", df_kauf.index.max())

🕓 Zeiträume der vier Datensätze:
Zinsen:      2009-05-01 00:00:00 → 2024-12-01 00:00:00
Inflation:   2013-01-01 00:00:00 → 2023-01-01 00:00:00
Wohnen:      2017-01-01 00:00:00 → 2024-10-01 00:00:00
Kaufpreis:   2018-03-31 00:00:00 → 2025-03-31 00:00:00


In [5]:
# 🔁 Schritt 3: Gemeinsamen Zeitraum der vier Datensätze berechnen

# Startdatum = spätestes Startdatum aller vier Datensätze
start_datum = max(df_zins.index.min(), df_inflation.index.min(), df_wohnen.index.min(), df_kauf.index.min())

# Enddatum = frühestes Enddatum aller vier Datensätze
end_datum = min(df_zins.index.max(), df_inflation.index.max(), df_wohnen.index.max(), df_kauf.index.max())

# Kontrolle: Ausgabe des Zeitraums
print("📆 Gemeinsamer Zeitraum für den Merge:")
print("⏳", start_datum.date(), "bis", end_datum.date())

📆 Gemeinsamer Zeitraum für den Merge:
⏳ 2018-03-31 bis 2023-01-01


In [6]:
# ✂️ Schritt 4: Datensätze auf den gemeinsamen Zeitraum beschränken

# Alle vier Datensätze auf Zeitraum zuschneiden
df_zins = df_zins[(df_zins.index >= start_datum) & (df_zins.index <= end_datum)]
df_inflation = df_inflation[(df_inflation.index >= start_datum) & (df_inflation.index <= end_datum)]
df_wohnen = df_wohnen[(df_wohnen.index >= start_datum) & (df_wohnen.index <= end_datum)]
df_kauf = df_kauf[(df_kauf.index >= start_datum) & (df_kauf.index <= end_datum)]

# Kontrolle: Zeiträume erneut anzeigen
print("📋 Zeiträume nach dem Zuschnitt:")
print("Zinsen:     ", df_zins.index.min().date(), "→", df_zins.index.max().date())
print("Inflation:  ", df_inflation.index.min().date(), "→", df_inflation.index.max().date())
print("Wohnen:     ", df_wohnen.index.min().date(), "→", df_wohnen.index.max().date())
print("Kaufpreis:  ", df_kauf.index.min().date(), "→", df_kauf.index.max().date())


📋 Zeiträume nach dem Zuschnitt:
Zinsen:      2018-04-01 → 2023-01-01
Inflation:   2019-01-01 → 2023-01-01
Wohnen:      2018-04-01 → 2023-01-01
Kaufpreis:   2018-03-31 → 2022-12-31


In [7]:
# 🔗 Schritt 5: Datensätze anhand des Zeitindexes mergen
# Ziel: Ein gemeinsamer DataFrame mit allen Variablen
# Methode: Inner Join → nur gemeinsame Zeitpunkte

df_merged = df_zins.join(df_inflation, how="inner") \
                   .join(df_wohnen, how="inner") \
                   .join(df_kauf, how="inner")

# ✅ Kontrolle: Ergebnis anzeigen
print("✅ Anzahl gemeinsamer Zeitpunkte:", len(df_merged))
df_merged.head()

✅ Anzahl gemeinsamer Zeitpunkte: 0


,Zins_Fest_MW,festhypo_5_7_median,festhypo_5_7_anzahl_abschluesse,Zins_Var_MW,variabelhypo_5_7_median,variabelhypo_5_7_anzahl_abschluesse,LIK,HVPI,totalindex_und_subindizes,wohneigentum_total,...,einfamilienhaeuser_gemeindetyp 3,einfamilienhaeuser_gemeindetyp 4,einfamilienhaeuser_gemeindetyp 5,eigentumswohnungen_egw,eigentumswohnungen_gemeindetyp 1,eigentumswohnungen_gemeindetyp 2,eigentumswohnungen_gemeindetyp 3,eigentumswohnungen_gemeindetyp 4,eigentumswohnungen_gemeindetyp 5,price_calculated


In [8]:
# 🛠 Zeitindex auf Quartalsbeginn normieren
# Wir wandeln den Index in Quartale um und setzen ihn auf den Beginn des Quartals
df_zins.index = df_zins.index.to_period("Q").to_timestamp()

In [9]:
df_inflation.index = df_inflation.index.to_period("Q").to_timestamp()

In [10]:
df_wohnen.index = df_wohnen.index.to_period("Q").to_timestamp()

In [11]:
df_kauf.index = df_kauf.index.to_period("Q").to_timestamp()

In [12]:
# 🔗 Schritt 5: Datensätze anhand des Zeitindexes mergen
# Ziel: Ein gemeinsamer DataFrame mit allen Variablen
# Methode: Inner Join → nur gemeinsame Zeitpunkte

df_merged = df_zins.join(df_inflation, how="inner") \
                   .join(df_wohnen, how="inner") \
                   .join(df_kauf, how="inner")

# ✅ Kontrolle: Ergebnis anzeigen
print("✅ Anzahl gemeinsamer Zeitpunkte:", len(df_merged))
df_merged.head()

✅ Anzahl gemeinsamer Zeitpunkte: 12


,Zins_Fest_MW,festhypo_5_7_median,festhypo_5_7_anzahl_abschluesse,Zins_Var_MW,variabelhypo_5_7_median,variabelhypo_5_7_anzahl_abschluesse,LIK,HVPI,totalindex_und_subindizes,wohneigentum_total,...,einfamilienhaeuser_gemeindetyp 3,einfamilienhaeuser_gemeindetyp 4,einfamilienhaeuser_gemeindetyp 5,eigentumswohnungen_egw,eigentumswohnungen_gemeindetyp 1,eigentumswohnungen_gemeindetyp 2,eigentumswohnungen_gemeindetyp 3,eigentumswohnungen_gemeindetyp 4,eigentumswohnungen_gemeindetyp 5,price_calculated
2019-01-01,1.2196,1.10,404,0.92384,0.90,99,0.4,0.4,2019Q1,97.6275,...,96.2836,98.8768,97.1565,98.1662,95.8170,98.8719,98.7477,100.5461,98.7016,825309.097848
2019-01-01,1.1661,1.10,310,0.92389,0.90,64,0.4,0.4,2019Q1,97.6275,...,96.2836,98.8768,97.1565,98.1662,95.8170,98.8719,98.7477,100.5461,98.7016,825309.097848
2019-01-01,1.1481,1.10,390,0.92855,0.90,62,0.4,0.4,2019Q1,97.6275,...,96.2836,98.8768,97.1565,98.1662,95.8170,98.8719,98.7477,100.5461,98.7016,825309.097848
2020-01-01,1.0769,1.04,369,0.86446,0.85,92,-0.7,-0.8,2020Q1,99.2443,...,98.7522,98.9907,99.1911,99.0520,97.5903,99.3591,98.0392,101.2635,99.1739,795583.306735
2020-01-01,1.0740,1.00,236,0.84412,0.85,34,-0.7,-0.8,2020Q1,99.2443,...,98.7522,98.9907,99.1911,99.0520,97.5903,99.3591,98.0392,101.2635,99.1739,795583.306735


In [13]:
# 📁 Export des finalen DataFrames zur lokalen Wiederverwendung
from google.colab import files

# CSV-Datei speichern
df_merged.to_csv("merged_cleaned.csv")

# Datei herunterladen
files.download("merged_cleaned.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>